In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import sys
sys.path.extend([#'../input/tf2_0_baseline_w_bert/',#'../input/bert_modeling/',
                 '../input/bert-joint-baseline/'])
import bert_utils
import modeling 
#import bert_optimization as optimization
import tokenization
import json

#tf.compat.v1.disable_eager_execution()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# In this case, we've got some extra BERT model files under `/kaggle/input/bertjointbaseline`

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
on_kaggle_server = os.path.exists('/kaggle')
nq_test_file = '../input/tensorflow2-question-answering/simplified-nq-test.jsonl' 
public_dataset = os.path.getsize(nq_test_file)<20_000_000
private_dataset = os.path.getsize(nq_test_file)>=20_000_000

print(on_kaggle_server,public_dataset,private_dataset)

In [ ]:
if True:
    import importlib
    importlib.reload(bert_utils)

In [ ]:
with open('../input/bert-joint-baseline/bert_config.json','r') as f:
    config = json.load(f)
print(json.dumps(config,indent=4))


In [ ]:
class TDense(tf.keras.layers.Layer):
    def __init__(self,
                 output_size,
                 kernel_initializer=None,
                 bias_initializer="zeros",
                **kwargs):
        super().__init__(**kwargs)
        self.output_size = output_size
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer

    def build(self,input_shape):
        dtype = tf.as_dtype(self.dtype or tf.keras.backend.floatx())
        if not (dtype.is_floating or dtype.is_complex):
          raise TypeError("Unable to build `TDense` layer with "
                          "non-floating point (and non-complex) "
                          "dtype %s" % (dtype,))
        input_shape = tf.TensorShape(input_shape)
        if tf.compat.dimension_value(input_shape[-1]) is None:
          raise ValueError("The last dimension of the inputs to "
                           "`TDense` should be defined. "
                           "Found `None`.")
        last_dim = tf.compat.dimension_value(input_shape[-1])
        self.input_spec = tf.keras.layers.InputSpec(min_ndim=2, axes={-1: last_dim})
        self.kernel = self.add_weight(
            "kernel",
            shape=[self.output_size,last_dim],
            initializer=self.kernel_initializer,
            dtype=self.dtype,
            trainable=True)
        self.bias = self.add_weight(
            "bias",
            shape=[self.output_size],
            initializer=self.bias_initializer,
            dtype=self.dtype,
            trainable=True)
        super(TDense, self).build(input_shape)
    def call(self,x):
        return tf.matmul(x,self.kernel,transpose_b=True)+self.bias
    
    def get_config(self):

        cfg = super().get_config().copy()
        cfg.update({
            'output_size': self.output_size,
            'kernel_initializer': self.kernel_initializer,
            'bias_initializer': self.bias_initializer,
        })
        return cfg
def mk_model(config):
    seq_len = config['max_position_embeddings']
    unique_id  = tf.keras.Input(shape=(1,),dtype=tf.int64,name='unique_id')
    input_ids   = tf.keras.Input(shape=(seq_len,),dtype=tf.int32,name='input_ids')
    input_mask  = tf.keras.Input(shape=(seq_len,),dtype=tf.int32,name='input_mask')
    segment_ids = tf.keras.Input(shape=(seq_len,),dtype=tf.int32,name='segment_ids')
    BERT = modeling.BertModel(config=config,name='bert')
    pooled_output, sequence_output = BERT(input_word_ids=input_ids,
                                          input_mask=input_mask,
                                          input_type_ids=segment_ids)
    
    logits = TDense(2,name='logits')(sequence_output)
    start_logits,end_logits = tf.split(logits,axis=-1,num_or_size_splits= 2,name='split')
    start_logits = tf.squeeze(start_logits,axis=-1,name='start_squeeze')
    end_logits   = tf.squeeze(end_logits,  axis=-1,name='end_squeeze')
    
    ans_type      = TDense(5,name='ans_type')(pooled_output)
    return tf.keras.Model([input_ for input_ in [unique_id,input_ids,input_mask,segment_ids] 
                           if input_ is not None],
                          [unique_id,start_logits,end_logits,ans_type],
                          name='bert-baseline')

In [ ]:
model= mk_model(config)

In [ ]:
model.summary()

In [ ]:
model.load_weights('../input/unk0201128w/weights')

In [ ]:
import tqdm
eval_records = "../input/bert-joint-baseline/nq-test.tfrecords"
#nq_test_file = '../input/tensorflow2-question-answering/simplified-nq-test.jsonl'
if private_dataset:
    eval_records='nq-test.tfrecords'
if not os.path.exists(eval_records):
    # tf2baseline.FLAGS.max_seq_length = 512
    eval_writer = bert_utils.FeatureWriter(
        filename=os.path.join(eval_records),
        is_training=False)

    tokenizer = tokenization.FullTokenizer(vocab_file='../input/bert-joint-baseline/vocab-nq.txt', 
                                           do_lower_case=True)

    features = []
    convert = bert_utils.ConvertExamples2Features(tokenizer=tokenizer,
                                                   is_training=False,
                                                   output_fn=eval_writer.process_feature,
                                                   collect_stat=False)

    n_examples = 0
    tqdm_notebook= tqdm.tqdm_notebook if not on_kaggle_server else None
    for examples in bert_utils.nq_examples_iter(input_file=nq_test_file, 
                                           is_training=False,
                                           tqdm=tqdm_notebook):
        for example in examples:
            n_examples += convert(example)

    eval_writer.close()
    print('number of test examples: %d, written to file: %d' % (n_examples,eval_writer.num_features))

In [ ]:
seq_length = bert_utils.FLAGS.max_seq_length #config['max_position_embeddings']
name_to_features = {
      "unique_id": tf.io.FixedLenFeature([], tf.int64),
      "input_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.io.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
  }

def _decode_record(record, name_to_features=name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.io.parse_single_example(serialized=record, features=name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
        t = example[name]
        if name != 'unique_id': #t.dtype == tf.int64:
            t = tf.cast(t, dtype=tf.int32)
        example[name] = t

    return example

def _decode_tokens(record):
    return tf.io.parse_single_example(serialized=record, 
                                      features={
                                          "unique_id": tf.io.FixedLenFeature([], tf.int64),
                                          "token_map" :  tf.io.FixedLenFeature([seq_length], tf.int64)
                                      })

In [ ]:
raw_ds = tf.data.TFRecordDataset(eval_records)
token_map_ds = raw_ds.map(_decode_tokens)
decoded_ds = raw_ds.map(_decode_record)
ds = decoded_ds.batch(batch_size=16,drop_remainder=False)

In [ ]:
%time result=model.predict_generator(ds,verbose=1 if not on_kaggle_server else 0)

In [ ]:
model.save('model.h5')  # 存檔

In [ ]:
import collections
Span = collections.namedtuple("Span", ["start_token_idx", "end_token_idx"])


class EvalExample(object):
  """Eval data available for a single example."""

  def __init__(self, example_id, candidates):
    self.example_id = example_id
    self.candidates = candidates
    self.results = {}
    self.features = {}


class ScoreSummary(object):

  def __init__(self):
    self.predicted_label = None
    self.short_span_score = None
    self.cls_token_score = None
    self.answer_type_logits = None

def get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(
      enumerate(logits[1:], 1), key=lambda x: x[1], reverse=True)
  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes


def compute_predictions(example):
  """Converts an example into an NQEval object for evaluation."""
  predictions = []
  n_best_size = 10
  max_answer_length = 30

  i = 0
  for unique_id, result in example.results.items():
    if unique_id not in example.features:
      raise ValueError("No feature found with unique_id:", unique_id)
    token_map = np.array(example.features[unique_id]["token_map"])
    start_indexes = get_best_indexes(result["start_logits"], n_best_size)
    end_indexes = get_best_indexes(result["end_logits"], n_best_size)
    for start_index in start_indexes:
      for end_index in end_indexes:
        if end_index < start_index:
          continue
        if token_map[start_index] == -1:
          continue
        if token_map[end_index] == -1:
          continue
        length = end_index - start_index + 1
        if length > max_answer_length:
          continue
        summary = ScoreSummary()
        summary.short_span_score = (
            result["start_logits"][start_index] +
            result["end_logits"][end_index])
        summary.cls_token_score = (
            result["start_logits"][0] + result["end_logits"][0])
        summary.answer_type_logits = result["answer_type_logits"]
        
        # additional scores 
        summary.start_score = result["start_logits"][start_index] 
        summary.end_score = result["end_logits"][end_index] 
        summary.cls_start_score = result["start_logits"][0]
        summary.cls_end_score = result["end_logits"][0]
        
        start_span = token_map[start_index]
        end_span = token_map[end_index] + 1

        # Span logits minus the cls logits seems to be close to the best.
        score = summary.short_span_score - summary.cls_token_score
        predictions.append((score, i, summary, start_span, end_span))
        i += 1

  short_span = Span(-1, -1)
  long_span = Span(-1, -1)

  if predictions:
    score, _, summary, start_span, end_span = sorted(predictions, reverse=True)[0]
    short_span = Span(start_span, end_span)
    
    matching_candidates = []
    start = short_span.start_token_idx
    end = short_span.end_token_idx
    
    c_corr = None
    for c in example.candidates:

      if c["start_token"] <= start and c["end_token"] >= end:
        long_span = Span(c["start_token"], c["end_token"])
        matching_candidates.append(c)
        
    if len(matching_candidates) == 1:
        c_corr = matching_candidates[0]
        if not c_corr['top_level']:
            c_corr = None
        
    if len(matching_candidates) > 2:
        for c in matching_candidates:
          if c['top_level']:
            c_corr = c
            break
            
    if len(matching_candidates) == 2:
        for c in matching_candidates:
          if not c['top_level'] and c['start_token_value'] == '<Li>':
            c_corr = c
        if not c_corr:
          for c in matching_candidates:
            if c['top_level'] :
                c_corr = c
    
    if c_corr:
        long_span = Span(c_corr["start_token"], c_corr["end_token"])
    
    
  else:
      summary = ScoreSummary()

      summary.predicted_label = {
      "example_id": int(example.example_id),
      "long_answer": {
          "start_token": int(-1),
          "end_token": int(-1),
          "start_byte": int(-1),
          "end_byte": int(-1)
      },
      "long_answer_score": float(-10),
      "short_answers": [],
      "short_answers_score": float(-10),
      "yes_no_answer": "NONE",
      "answer_type_logits": [10, -10, -10, -10, -10],
      "answer_type": int(0),
          
      # additional scores
      "start_score": float(-100), 
      "end_score": float(-100),
      "cls_start_score": float(100),
      "cls_end_score": float(100)
      }
      return summary
        
  summary.predicted_label = {
      "example_id": int(example.example_id),
      "long_answer": {
          "start_token": int(long_span.start_token_idx),
          "end_token": int(long_span.end_token_idx),
          "start_byte": -1,
          "end_byte": -1
      },
      "long_answer_score": float(summary.start_score * 0.3 + summary.end_score * 0.15 - 0.4 * summary.cls_start_score - 0.4 * summary.cls_end_score),
      "short_answers": [{
          "start_token": int(short_span.start_token_idx),
          "end_token": int(short_span.end_token_idx),
          "start_byte": -1,
          "end_byte": -1
      }],
      "short_answers_score": float(score * 0.22 -0.1 * summary.answer_type_logits[0] + 0.5 *  summary.answer_type_logits[3] - 0.3 * summary.answer_type_logits[4]),
      "yes_no_answer": "NONE",
      "answer_type_logits": summary.answer_type_logits.tolist(),
      "answer_type": int(np.argmax(summary.answer_type_logits)),
      # additional scores
      "start_score": float(summary.start_score), 
      "end_score": float(summary.end_score),
      "cls_start_score": float(summary.cls_start_score),
      "cls_end_score": float(summary.cls_end_score)
  }

  return summary

def compute_pred_dict(candidates_dict, dev_features, raw_results,tqdm=None):
    """Computes official answer key from raw logits."""
    raw_results_by_id = [(int(res['unique_id']),1, res) for res in raw_results]

    examples_by_id = [(int(k),0,v) for k, v in candidates_dict.items()]
  
    features_by_id = [(int(d['unique_id']),2,d) for d in dev_features] 
  
    # Join examples with features and raw results.
    examples = []
    print('merging examples...')
    merged = sorted(examples_by_id + raw_results_by_id + features_by_id)
    print('done.')
    for idx, type_, datum in merged:
        if type_==0: #isinstance(datum, list):
            examples.append(EvalExample(idx, datum))
        elif type_==2: #"token_map" in datum:
            examples[-1].features[idx] = datum
        else:
            examples[-1].results[idx] = datum

    # Construct prediction objects.
    print('Computing predictions...')
   
    nq_pred_dict = {}
    if tqdm is not None:
        examples = tqdm(examples)
    for e in examples:
        summary = compute_predictions(e)
        nq_pred_dict[e.example_id] = summary.predicted_label

    return nq_pred_dict

In [ ]:
def read_candidates_from_one_split(input_path):
  """Read candidates from a single jsonl file."""
  candidates_dict = {}
  print("Reading examples from: %s" % input_path)
  if input_path.endswith(".gz"):
    with gzip.GzipFile(fileobj=tf.io.gfile.GFile(input_path, "rb")) as input_file:
      for index, line in enumerate(input_file):
        e = json.loads(line)
        candidates = e["long_answer_candidates"]
        tokens = e['document_text'].split()
        add_candidates = []
        for c in candidates:
            c_start_token = tokens[c['start_token']]
            c['start_token_value'] = c_start_token
            add_candidates.append(c)
        candidates_dict[e["example_id"]] = add_candidates
        
  else:
    with tf.io.gfile.GFile(input_path, "r") as input_file:
      for index, line in enumerate(input_file):
        e = json.loads(line)
        candidates = e["long_answer_candidates"]
        tokens = e['document_text'].split()
        add_candidates = []
        for c in candidates:
            c_start_token = tokens[c['start_token']]
            c['start_token_value'] = c_start_token
            add_candidates.append(c)
        candidates_dict[e["example_id"]] = add_candidates
  return candidates_dict


def read_candidates(input_pattern):
  """Read candidates with real multiple processes."""
  input_paths = tf.io.gfile.glob(input_pattern)
  final_dict = {}
  for input_path in input_paths:
    final_dict.update(read_candidates_from_one_split(input_path))
  return final_dict

In [ ]:
all_results = [bert_utils.RawResult(*x) for x in zip(*result)]
    
print ("Going to candidates file")

candidates_dict = read_candidates('../input/tensorflow2-question-answering/simplified-nq-test.jsonl')

print ("setting up eval features")

eval_features = list(token_map_ds)

print ("compute_pred_dict")

tqdm_notebook= tqdm.tqdm_notebook if not on_kaggle_server else None
nq_pred_dict = compute_pred_dict(candidates_dict, 
                                       eval_features,
                                       [r._asdict() for r in all_results])

predictions_json = {"predictions": list(nq_pred_dict.values())}

print ("writing json")

with tf.io.gfile.GFile('predictions.json', "w") as f:
    json.dump(predictions_json, f, indent=4)

In [ ]:
def create_short_answer(entry):
    
    if entry['answer_type'] == 1:
        return 'YES'
        
    if entry['answer_type'] == 2:
        return 'NO'
    
    
    if entry["short_answers_score"] < 1.75:
        return ""

    
    if entry['answer_type'] == 4 : #or single_prediction['answer_type'] == 0 :
        return ""
    
    
    answer = []    
    for short_answer in entry["short_answers"]:
        if short_answer["start_token"] > -1:
            answer.append(str(short_answer["start_token"]) + ":" + str(short_answer["end_token"]))
    if entry["yes_no_answer"] != "NONE":
        answer.append(entry["yes_no_answer"])
    return " ".join(answer)

def create_long_answer(entry):
    if entry["long_answer_score"] < -2.85:
        return ""

    answer = []
    if entry["long_answer"]["start_token"] > -1:
        answer.append(str(entry["long_answer"]["start_token"]) + ":" + str(entry["long_answer"]["end_token"]))
    return " ".join(answer)

In [ ]:
test_answers_df = pd.read_json("../working/predictions.json")
for var_name in ['long_answer_score','short_answers_score','answer_type']:
    test_answers_df[var_name] = test_answers_df['predictions'].apply(lambda q: q[var_name])
test_answers_df["long_answer"] = test_answers_df["predictions"].apply(create_long_answer)
test_answers_df["short_answer"] = test_answers_df["predictions"].apply(create_short_answer)
test_answers_df["example_id"] = test_answers_df["predictions"].apply(lambda q: str(q["example_id"]))

long_answers = dict(zip(test_answers_df["example_id"], test_answers_df["long_answer"]))
short_answers = dict(zip(test_answers_df["example_id"], test_answers_df["short_answer"]))

In [ ]:
sample_submission = pd.read_csv("../input/tensorflow2-question-answering/sample_submission.csv")

long_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_long")].apply(lambda q: long_answers[q["example_id"].replace("_long", "")], axis=1)
short_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_short")].apply(lambda q: short_answers[q["example_id"].replace("_short", "")], axis=1)

sample_submission.loc[sample_submission["example_id"].str.contains("_long"), "PredictionString"] = long_prediction_strings
sample_submission.loc[sample_submission["example_id"].str.contains("_short"), "PredictionString"] = short_prediction_strings

In [ ]:
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
sample_submission.head(10)

In [ ]:
if public_dataset:
    print(test_answers_df["long_answer_score"].describe())
    print("-------------------")
    print(test_answers_df["short_answers_score"].describe())

In [ ]:
if public_dataset:
    print(np.bincount(test_answers_df['answer_type'].values))

In [ ]:
if public_dataset:
    print(test_answers_df[test_answers_df['answer_type']==0])

In [ ]:
if public_dataset:
    print(test_answers_df.predictions.values[-4])

In [ ]:
if public_dataset:
    print(sample_submission.head(10))

In [ ]:
class ShowPrediction:
    def __init__(self,jsonl_file):
        self._data = {}
        with open(jsonl_file,'r') as f:
            for line in f.readlines():
                d = json.loads(line)
                #print(d.keys())
                self._data[int(d['example_id'])]={
                    'text': d['document_text'],
                    'question': d['question_text']
                }
    def __call__(self,prediction,include_full_text=True):
        data = self._data[prediction['example_id']]
        res = {'question': data['question']}
        if include_full_text:
            res['text'] = data['text']
        for type_ in ['long_answer','short_answers']:
            ans = prediction[type_]
            if isinstance(ans,list):
                ans = ans[0]
            start,end = ans['start_token'],ans['end_token']
            res[type_] = ' '.join(data['text'].split()[start:end])
        return res

In [ ]:
if public_dataset:
    show_pred = ShowPrediction('../input/tensorflow2-question-answering/simplified-nq-test.jsonl')

In [ ]:
if public_dataset:
    for pred in test_answers_df.predictions[test_answers_df.answer_type==0]:
        print(json.dumps(show_pred(pred,include_full_text=True),indent=4))
        sys.exit()

In [ ]:
if public_dataset:
    for pred in np.random.choice(predictions_json['predictions'],10):
        print(json.dumps(show_pred(pred,include_full_text=False),indent=4))